<h1><font color="#113D68" size=5>Deep Learning para Procesamiento del Lenguaje Natural</font></h1>



<h1><font color="#113D68" size=6>BERT: Clasificación de texto<font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Extraído de la asignatura de Deep Learning para el Procesamiento del Lenguaje Natural del Máster Universitario en Inteligencia Artificial de la UPM</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Procesamiento de datos](#section1)
* [2. Procesamiento en Hugging Face](#section2)
* [3. Finetuning para clasificación de texto](#section3)

<a id="section0"></a>
# <font color="#004D7F" size=6>0. Contexto</font>

Para este ejemplo, utilizaremos el mismo corpus compuesto por tramas de películas. Sin embargo, para esta tarea solo estamos empleando un subconjunto de tramas de películas: solo aquellas que corresponden a películas de comedia, drama u western. 

Por lo tanto, nuestro objetivo en este ejemplo es clasificar las tramas de las películas en estos tres géneros.

---

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section0"></a>
# <font color="#004D7F" size=6>1. Procesamiento de datos</font>

Este archivo tiene el formato: 
```text
Plot | "tab" | Label 
```
Entonces, nuestro primer paso sería separar etiquetas y gráficos. Para esta tarea, utilizaremos la biblioteca pandas, ya que permite un procesamiento sencillo de csv:

In [2]:
import pandas as pd
with open('data/movie_plots_tc.csv',encoding='utf-8',errors='ignore') as csv_file:
  df=pd.read_csv(csv_file,sep=';')
plots=df['Plot']
labels=df['Genre']

De todas las muestras existentes, seleccionamos una submuestra de 500 elementos para efectos de cómputo

In [3]:
plots=plots[:500]
labels=labels[:500]

Las redes neuronales no pueden predecir sobre etiquetas de tipo cadena. Como este es un problema de clasificación de múltiples clases, necesitamos codificar nuestras etiquetas en un formato numérico.

In [4]:
import numpy as np
str2id={'western':0,'drama':1,'comedy':2}
id2str={0:'western',1:'drama',2:'comedy'}

list_plots=plots.fillna("CVxTz").values
indexed_labels=np.array([str2id[l] for l in labels])


Como de costumbre, necesitamos dividir nuestros datos entre dos conjuntos: entrenamiento y validación.

In [5]:
from sklearn.model_selection import train_test_split
train_features, val_features, train_labels, val_labels = train_test_split(list_plots, indexed_labels, test_size=0.25, random_state=2018)

Instalación de la biblioteca de transformadores para usar HuggingFace

---

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Procesamiento en Hugging Face</font>

In [ ]:
!pip3 install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 40.4 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 6.5 MB 38.6 MB/s 
     |████████████████████████████████| 895 kB 32.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Cargamos y procesamos datos de Hugging Face

In [6]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

2022-12-07 15:57:41.547918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 15:57:44.595306: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/include:/usr/local/cuda-11.0/lib64:
2022-12-07 15:57:44.595638: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/include:/usr/local/cuda-11.0/lib64:
2022-12-07 15:57

Para esta sesión, usaremos el modelo `bert-base-uncased`, disponible en Hugging Face. Se establece una longitud máxima de 256 tokens por muestra

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el [modelo](https://huggingface.co/bert-base-uncased)

In [7]:
model_name= "bert-base-uncased"
max_length=256
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

Ahora tokenizamos nuestras muestras de texto de entrenamiento y validación 

__¡Cuidado!__ Las muestras de entrada deben ser `str` o `List[str]`.

In [8]:
train_encodings = tokenizer(train_features.tolist(),truncation=True,padding=True,max_length=max_length)
val_encodings = tokenizer(val_features.tolist(),truncation=True,padding=True,max_length=max_length)

Ahora podemos construirlo en Torch. Conjunto de datos utilizando las codificaciones calculadas de la siguiente manera:

In [9]:
class OurTorchDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [10]:
train_dataset=OurTorchDataset(train_encodings,train_labels)
val_dataset=OurTorchDataset(val_encodings,val_labels)

De forma predeterminada, HuggingFace no calcula automáticamente las métricas que estamos buscando. Necesitamos definir una función personalizada para calcular el Accuracy.

In [11]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

---

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Finetuning para clasificación de texto</font>

Ahora podemos cargar un modelo básico preentrenado de HuggingFace. ¡Asegúrese de especificar el número correcto de etiquetas!

In [12]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3).to("cuda")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

We

In [13]:
training_args = TrainingArguments(
    output_dir='.',     # output directory
    num_train_epochs=3,              # total number of training epochs  # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  
    seed=1895,             # strength of weight decay
)

In [14]:
trainer = Trainer(
    model=model, 
    args= training_args,                        # the instantiated Transformers model to be trained               # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset, 
    compute_metrics=compute_metrics # the callback that computes metrics of interest
)
trainer.train()

/home/manwest/Documentos/Jupyter/jupyterEnv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 375
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 141


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=141, training_loss=0.1003651517502805, metrics={'train_runtime': 271.3006, 'train_samples_per_second': 4.147, 'train_steps_per_second': 0.52, 'total_flos': 148001297472000.0, 'train_loss': 0.1003651517502805, 'epoch': 3.0})

In [15]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 125
  Batch size = 8


{'eval_loss': 0.00047253284719772637,
 'eval_accuracy': 1.0,
 'eval_runtime': 14.4204,
 'eval_samples_per_second': 8.668,
 'eval_steps_per_second': 1.11,
 'epoch': 3.0}

In [16]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return id2str[probs.argmax().item()]

In [17]:
get_prediction("The duo decide to search for the gold together, but they are apprehended by Union forces shortly after leaving the mission - Tuco yells out Confederate-supportive statements at a group of Union soldiers, as they are covered in dust, obscuring the blue color of their uniforms. The two are brought to a prison camp which Angel infiltrated as a Union sergeant in his search for Bill Carson, getting his attention when Tuco poses as Bill Carson. Tuco reveals the name of the cemetery under torture and is sent away to be killed. Knowing that Blondie would not reveal the location, Angel Eyes recruits him into his search. Tuco escapes his fate by killing Angel Eyes' henchman, and soon finds himself in an evacuated town, where Blondie, Angel Eyes, and his gang have also arrived. ")

'western'

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>